# Investment Visualization

ParkScore에 사용되는 지표 중 투자의 시각화를 진행한다.  
투자는 3년간의 도시의 총 예산 중 도시공원 관련 예산이 어느정도 차지하는 지 파악한다.

## Import

- 최종 ParkScore 점수 사용

In [32]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

In [33]:
try:
    invest = pd.read_csv('./investment.csv', encoding='utf-8')
except UnicodeDecodeError:
    try:
        invest = pd.read_csv('./investment.csv', encoding='cp949')
    except Exception as e:
        print(f"Error: {e}")

In [34]:
invest

,도,도시명,공원투자예산\n(2024),공원투자예산\n(2023),공원투자예산\n(2022),총예산\n(2024),총예산\n(2023),총예산\n(2022),도시공원투자비용 / 총비용,최종 점수\n(100점이상 100점으로 환산),순위,도시공원 관련 부서
0,광역특별시,대구광역시,"30,431,074","94,297,364","170,417,072","10,587,194,000","10,730,765,000","10,144,400,000",94.0,94.0,49.0,공원조성과과
1,광역특별시,광주광역시,"122,076,104","98,102,845","71,629,355","6,904,221,482","7,110,179,032","7,009,427,226",139.0,100.0,1.0,도시공원과
2,광역특별시,대전광역시,"32,390,842","39,816,691","41,133,719","6,532,974,000","6,561,725,000","6,365,186,000",58.0,58.0,68.0,공원관리사업소
3,광역특별시,부산광역시,"167,218,029","43,233,646","35,578,916","15,699,527,956","15,327,743,441","14,269,024,509",54.0,54.0,71.0,공원정책과
4,광역특별시,서울특별시,"445,413,563","545,292,134","556,695,518","45,740,517,556","47,190,512,274","44,219,048,802",113.0,100.0,1.0,푸른도시여가국
...,...,...,...,...,...,...,...,...,...,...,...,...
81,충청남도,천안시,"42,529,457","30,531,293","18,214,492","2,400,000,000","2,380,000,000","2,340,000,000",128.0,100.0,1.0,공원녹지과
82,충청북도,제천시,"18,762,397","26,017,503","20,723,413","1,082,742,095","1,059,504,466","1,006,658,145",208.0,100.0,1.0,산림공원과
83,충청북도,청주시,"19,549,934","23,508,926","12,980,644","3,239,120,962","3,284,202,861","3,009,322,462",59.0,50.0,73.0,공원관리과
84,충청북도,충주시,"3,141,531","9,053,668","17,124,556","1,388,501,162","1,303,412,422","1,243,708,369",74.0,74.0,58.0,생태건강도시과


## Data Preprocessing

In [35]:
# 필요한 열 제외 제거
invest = invest.drop(['도', '도시공원투자비용 / 총비용', '최종 점수\n(100점이상 100점으로 환산)', '순위', '도시공원 관련 부서'],
                    axis=1)

In [36]:
invest

,도시명,공원투자예산\n(2024),공원투자예산\n(2023),공원투자예산\n(2022),총예산\n(2024),총예산\n(2023),총예산\n(2022)
0,대구광역시,"30,431,074","94,297,364","170,417,072","10,587,194,000","10,730,765,000","10,144,400,000"
1,광주광역시,"122,076,104","98,102,845","71,629,355","6,904,221,482","7,110,179,032","7,009,427,226"
2,대전광역시,"32,390,842","39,816,691","41,133,719","6,532,974,000","6,561,725,000","6,365,186,000"
3,부산광역시,"167,218,029","43,233,646","35,578,916","15,699,527,956","15,327,743,441","14,269,024,509"
4,서울특별시,"445,413,563","545,292,134","556,695,518","45,740,517,556","47,190,512,274","44,219,048,802"
...,...,...,...,...,...,...,...
81,천안시,"42,529,457","30,531,293","18,214,492","2,400,000,000","2,380,000,000","2,340,000,000"
82,제천시,"18,762,397","26,017,503","20,723,413","1,082,742,095","1,059,504,466","1,006,658,145"
83,청주시,"19,549,934","23,508,926","12,980,644","3,239,120,962","3,284,202,861","3,009,322,462"
84,충주시,"3,141,531","9,053,668","17,124,556","1,388,501,162","1,303,412,422","1,243,708,369"


In [37]:
# '광주광역시' 추출
invest = invest[invest['도시명'] == '광주광역시']

In [38]:
# 필요없는 공백 제거 및 데이터 변환
invest.columns = [col.strip() for col in invest.columns]

In [39]:
# 연도별 데이터 추출 및 변환
park_budget_2024 = int(invest['공원투자예산\n(2024)'].values[0].replace(',', ''))
total_budget_2024 = int(invest['총예산\n(2024)'].values[0].replace(',', ''))

park_budget_2023 = int(invest['공원투자예산\n(2023)'].values[0].replace(',', ''))
total_budget_2023 = int(invest['총예산\n(2023)'].values[0].replace(',', ''))

park_budget_2022 = int(invest['공원투자예산\n(2022)'].values[0].replace(',', ''))
total_budget_2022 = int(invest['총예산\n(2022)'].values[0].replace(',', ''))

In [40]:
# 데이터 세팅
budgets = {
    "2024": [park_budget_2024, total_budget_2024 - park_budget_2024],
    "2023": [park_budget_2023, total_budget_2023 - park_budget_2023],
    "2022": [park_budget_2022, total_budget_2022 - park_budget_2022]
}

## EDA

- Pie 그래프를 이용하여 각 연도 별 총 예산 중 도시공원투자예산 확인 

In [41]:
# 서브플롯 생성
fig = make_subplots(rows=1, cols=3, specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=("2022", "2023", "2024"))

In [42]:
# 각 서브플롯에 파이 차트 추가
colors = ['#66AC00', '#ced4da'] 
fig.add_trace(go.Pie(labels=['도시공원 예산', '총 예산'], values=budgets["2022"], name="2022", 
                     marker=dict(colors=colors), hole=0.4), 1, 1)
fig.add_trace(go.Pie(labels=['도시공원 예산', '총 예산'], values=budgets["2023"], name="2023", 
                     marker=dict(colors=colors), hole=0.4), 1, 2)
fig.add_trace(go.Pie(labels=['도시공원 예산', '총 예산'], values=budgets["2024"], name="2024", 
                     marker=dict(colors=colors), hole=0.4), 1, 3)

In [43]:
# 레이아웃 업데이트
fig.update_layout(
    title_text="INVESTMENT (2022-2024)",
    annotations=[
        dict(text='광주광역시 예산<br>2022', x=0.145, y=0.5, font=dict(size=18,color='black'), showarrow=False, xanchor='center', yanchor='middle'),
        dict(text='광주광역시 예산<br>2023', x=0.5, y=0.5, font=dict(size=18,color='black'), showarrow=False, xanchor='center', yanchor='middle'),
        dict(text='광주광역시 예산<br>2024', x=0.855, y=0.5, font=dict(size=18,color='black'), showarrow=False, xanchor='center', yanchor='middle')
    ],
    font=dict(size=20)
)
fig.show()

In [44]:
# pie 그래프 저장
pio.write_html(fig, file='./investment_html/investment_pie.html')